<a href="https://colab.research.google.com/github/gabibu/nlp/blob/master/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2

This assignment is about training and evaluating a POS tagger with some real data. The dataset is available through the Universal Dependencies (https://universaldependencies.org/) (UD) project. To get to know the project, please visit https://universaldependencies.org/introduction.html)

In [1]:
import numpy as np
import operator
import nltk

!pip install conllutils
!pip install conll_df
import conllutils

  Created wheel for conllutils: filename=conllutils-1.1.4-cp37-none-any.whl size=17696 sha256=8a3f55d7545aa2d4248ca56d7e25cc1ad259d8e5408cef9a89bb0230bbb813c1
  Stored in directory: /root/.cache/pip/wheels/94/08/3a/74fab1fdde78b548f26b2de930af3cee81a592fafe9e758a57
Successfully built conllutils
  ERROR: Failed building wheel for conll-df
  Running setup.py clean for conll-df
Failed to build conll-df
    Running setup.py install for conll-df ... done


**Part 1** (getting the data)

You can download the dataset files directly from the UD website, but it will let you only download all the languages in one compressed file. In this assignment you will be working with th GUM dataset, which you can download directly from:
https://github.com/UniversalDependencies/UD_English-GUM.
Please download it to your colab machine.



In [2]:
!git clone https://github.com/UniversalDependencies/UD_English-GUM

Cloning into 'UD_English-GUM'...
remote: Enumerating objects: 2518, done.
remote: Counting objects: 100% (1529/1529), done.
remote: Compressing objects: 100% (660/660), done.
remote: Total 2518 (delta 1342), reused 1050 (delta 869), pack-reused 989
Receiving objects: 100% (2518/2518), 16.21 MiB | 9.84 MiB/s, done.
Resolving deltas: 100% (2232/2232), done.


We will use the (train/dev/test) files:

UD_English-GUM/en_gum-ud-train.conllu

UD_English-GUM/en_gum-ud-dev.conllu

UD_English-GUM/en_gum-ud-test.conllu

They are all formatted in the conllu format. You may read about it [here](https://universaldependencies.org/format.html). There is a utility library **conllutils**, which can help you read the data into the memory. It has already been installed and imported above.

You should write a code that reads the three datasets into memory. You may choose the data structure by yourself. As you can see, every word is represented by a line, with columns representing specific features. We are only interested in the first and fourth columns, corresponding to the word and its POS tag.

In [3]:
from conllutils import pipe

# Your code goes here

def read_dataset(file_path):
  
  p = pipe().read_conllu(file_path)
  return [[(t.form, t.upos) for t in s] for s in p.collect()]
  
train_file = "UD_English-GUM/en_gum-ud-train.conllu"
dev_file = 'UD_English-GUM/en_gum-ud-dev.conllu'
test_file = 'UD_English-GUM/en_gum-ud-test.conllu'

train_data = read_dataset(train_file)
dev_data = read_dataset(dev_file)
test_data = read_dataset(test_file)
len(train_data), len(dev_data), len(test_data), train_data[0:2]

(4287,
 784,
 890,
 [[('Aesthetic', 'ADJ'),
   ('Appreciation', 'NOUN'),
   ('and', 'CCONJ'),
   ('Spanish', 'ADJ'),
   ('Art', 'NOUN'),
   (':', 'PUNCT')],
  [('Insights', 'NOUN'), ('from', 'ADP'), ('Eye-Tracking', 'NOUN')]])

We will use the (train/dev/test) files:

UD_English-GUM/en_gum-ud-train.conllu

UD_English-GUM/en_gum-ud-dev.conllu

UD_English-GUM/en_gum-ud-test.conllu

They are all formatted in the conllu format. You may read about it [here](https://universaldependencies.org/format.html). There is a utility library **conllutils**, which can help you read the data into the memory. It has already been installed and imported above.

You should write a code that reads the three datasets into memory. You may choose the data structure by yourself. As you can see, every word is represented by a line, with columns representing specific features. We are only interested in the first and fourth columns, corresponding to the word and its POS tag.

We will use the (train/dev/test) files:

UD_English-GUM/en_gum-ud-train.conllu

UD_English-GUM/en_gum-ud-dev.conllu

UD_English-GUM/en_gum-ud-test.conllu

They are all formatted in the conllu format. You may read about it [here](https://universaldependencies.org/format.html). There is a utility library **conllutils**, which can help you read the data into the memory. It has already been installed and imported above.

You should write a code that reads the three datasets into memory. You may choose the data structure by yourself. As you can see, every word is represented by a line, with columns representing specific features. We are only interested in the first and fourth columns, corresponding to the word and its POS tag.

**Part 2**

Write a class **simple_tagger**, with methods *train* and *evaluate*. The method *train* receives the data as a list of sentences, and use it for training the tagger. In this case, it should learn a simple dictionary that maps words to tags, defined as the most frequent tag for every word (in case there is more than one most frequent tag, you may select one of them randomly). The dictionary should be stored as a class member for evaluation.

The method *evaluate* receives the data as a list of sentences, and use it to evaluate the tagger performance. Specifically, you should calculate the word and sentence level accuracy.
The evaluation process is simply going word by word, querying the dictionary (created by the train method) for each word’s tag and compare it to the true tag of that word. The word-level accuracy is the number of successes divided by the number of words. For OOV (out of vocabulary, or unknown) words, the tagger should assign the most frequent tag in the entire training set (i.e., the mode). The function should return the two numbers: word level accuracy and sentence level accuracy.


In [4]:
from collections import defaultdict
class simple_tagger:
  
  def __init__(self):
    self.mapper  = None

  def train(self, data):
    word_tag_counter = defaultdict(lambda : defaultdict(lambda : 0))
    tag_counter = defaultdict(lambda : 0)


    for sentense in data:
      for (word, tag) in sentense:
        word_tag_counter[word][tag] += 1
        tag_counter[tag] += 1
    
    default_tag = None
    default_tag_max = 0 

    for tag, count in tag_counter.items():
      if count > default_tag_max:
        default_tag_max = count
        default_tag = tag 
    
    self.mapper = defaultdict(lambda : default_tag)

    for word, tags_counter in word_tag_counter.items():
      max_count = 0
      best_word_tag = None
      for tag, counter in tags_counter.items():
        if counter > max_count:
          max_count = counter
          best_word_tag = tag
      
      self.mapper[word] = best_word_tag
  
  def evaluate(self, data):
    
    num_of_words = 0.0
    num_of_correct_words = 0.0
    num_of_correct_sentenses = 0.0 

    for sentense in data:
      num_of_words += len(sentense)
      correct_sentense = True
      for (word, annotated_tag) in sentense:
        model_tag = self.mapper[word]
        if model_tag == annotated_tag:
          num_of_correct_words += 1
        else:
          correct_sentense = False
      
      if correct_sentense:
        num_of_correct_sentenses += 1
    
    return num_of_correct_words/num_of_words, num_of_correct_sentenses/len(data)

**Part 3**

Similar to part 2, write the class hmm_tagger, which implements HMM tagging. The method *train* should build the matrices A, B and Pi, from the data as discussed in class. The method *evaluate* should find the best tag sequence for every input sentence using he Viterbi decoding algorithm, and then calculate the word and sentence level accuracy using the gold-standard tags. You should implement the Viterbi algorithm in the next block and call it from your class.

Additional guidance:
1. The matrix B represents the emissions probabilities. Since B is a matrix, you should build a dictionary that maps every unique word in the corpus to a serial numeric id (starting with 0). This way columns in B represents word ids.
2. During the evaluation, you should first convert each word into it’s index and then create the observation array to be given to Viterbi, as a list of ids. OOV words should be assigned with a random tag. To make sure Viterbi works appropriately, you can simply break the sentence into multiple segments every time you see an OOV word, and decode every segment individually using Viterbi.


In [30]:
class hmm_tagger:

    def __init__(self):
        self._all_tags = None
        self._tag_to_index = None
        self._initial_probs = None
        self._transition_matrix = None
        self._all_words = None
        self._word_to_index = None
        self._emission_matrix = None
        self._index_to_word = None
        self._index_to_tag = None

    def train(self, data):
        self._build_transion_matrix(data)
        self._build_emission_matrix(data)

    def _build_emission_matrix(self, data):
        self._all_words = {word_tag[0] for sentense in data for word_tag in sentense}
        self._word_to_index = {word: i for (i, word) in enumerate(self._all_words)}
        self._index_to_word = {i: word for (word, i) in self._word_to_index.items()}

        emission_counts = np.zeros(shape=(len(self._all_tags), len(self._all_words)))

        for sentense in data:
            for (word, tag) in sentense:
                word_index = self._word_to_index[word]
                tag_index = self._tag_to_index[tag]

                emission_counts[tag_index][word_index] += 1

        rows_count = np.sum(emission_counts, axis=1)
        self._emission_matrix = (emission_counts.T / rows_count).T

    def _build_transion_matrix(self, data):
        self._all_tags = {word_tag[1] for sentense in data for word_tag in sentense}
        self._tag_to_index = {tag: i for (i, tag) in enumerate(self._all_tags)}
        self._index_to_tag = {index: tag for (tag, index) in self._tag_to_index.items()}

        initial_counts = np.zeros(shape=len(self._all_tags))

        a_counts = np.zeros(shape=(len(self._all_tags), len(self._all_tags)))

        for sentense in data:
            prev_tag = None
            for word_index, (word, tag) in enumerate(sentense):
                # if word_index == 0:
                initial_counts[self._tag_to_index[tag]] += 1
                # else:
                if word_index > 0:
                    a_counts[self._tag_to_index[prev_tag]][self._tag_to_index[tag]] += 1

                prev_tag = tag

        self._initial_probs = initial_counts / np.sum(initial_counts)

        rows_count = np.sum(a_counts, axis=1)
        self._transition_matrix = (a_counts.T / rows_count).T

    def tagdata(self, data):

        results = []
        for sentense in data:
            words_ids = [self._word_to_index[word_tag[0]] for word_tag in sentense]
            res = viterbi(words_ids, self._transition_matrix, self._emission_matrix, self._initial_probs)
            results.append(
                [(self._index_to_word[word_id], self._index_to_tag[tag]) for (word_id, tag) in zip(words_ids, res)])

        return results

    def _tag(self, sentense):

        words_ids = [self._word_to_index[word_tag[0]] if word_tag[0] in self._word_to_index else None for word_tag in
                     sentense]

        start_index = 0
        end_index = start_index
        res = []
        while end_index < len(words_ids):
            if words_ids[end_index] is not None:
                end_index += 1
            else:
                broken_word_ids = words_ids[start_index: end_index]
                if len(broken_word_ids) > 0:
                    broken_res = viterbi(broken_word_ids, self._transition_matrix, self._emission_matrix, self._initial_probs)
                    res.extend(broken_res)

                possible_tags_ids = list(self._tag_to_index.values())
                unkown_word_tag_id = np.random.choice(len(possible_tags_ids))
                res.append(unkown_word_tag_id)
                start_index = end_index + 1
                end_index = start_index

        if start_index < len(words_ids):
            broken_word_ids = words_ids[start_index:]
            broken_res = viterbi(broken_word_ids, self._transition_matrix, self._emission_matrix, self._initial_probs)
            res.extend(broken_res)

        return res

    def evaluate(self, data):

        correct_sentense_counter = 0.0
        correct_tags_counter = 0.0
        num_of_words = 0.0

        for sentense in data:
            res = self._tag(sentense)

            expected_tags_ids = [self._tag_to_index[word_tag[1]] for word_tag in sentense]
            correct_sentense = True
            num_of_words += len(expected_tags_ids)

            matched_arr = [1 for (expected, predicted) in zip(res, expected_tags_ids) if expected == predicted]

            correct_tags_counter += len(matched_arr)
            correct_sentense_counter += 1 if len(res) == len(matched_arr) else 0

        return correct_tags_counter / num_of_words, correct_sentense_counter / len(data)

In [9]:
# Viterbi
def viterbi (observations, A, B, Pi):
  #...

  N = A.shape[0]
  lam = np.zeros(shape=(N, len(observations)))
  arm_max_vals = np.zeros(shape=(N, len(observations)))
  #init step 

  #for o in observations:
  for n in range(0, N):
    lam[n][0] = B[n][observations[0]] * Pi[n] 
    arm_max_vals[n][0] = 0
  
  #iteration step 
  for t in range(1, len(observations)):
    for n in range(0, N):
      max_before_prob = -1
      max_before_tag = None
      for j in range(0, N):
        current_prev_prob = lam[j][t-1] * A[j][n]

        if current_prev_prob > max_before_prob:
          max_before_prob = current_prev_prob
          max_before_tag = j 
      lam[n][t] = B[n][observations[t]] * max_before_prob
      arm_max_vals[n][t] = max_before_tag

  best_sequence = np.zeros(shape =len(observations), dtype =np.int32)

  
  best_sequence[-1] = best_sequence[-1] = np.argmax(lam[:, len(observations) -1]) #arm_max_vals[np.argmax(lam[:, len(observations)-1]), len(observations)-1]
  
  for t in range(len(observations)-2, -1, -1):
    best_sequence[t] = arm_max_vals[best_sequence[t +1]][t+1]

  return best_sequence


  #return best_sequence

# A simple example to run the Viterbi algorithm:
#( Same as in presentation "NLP 3 - Tagging" on slide 35)

A = np.array([[0.3, 0.7], [0.2, 0.8]])
B = np.array([[0.1, 0.1, 0.3, 0.5], [0.3, 0.3, 0.2, 0.2]])
Pi = np.array([0.4, 0.6])
print(viterbi([0, 3, 1, 0], A, B, Pi))
# Expected output: 1, 1, 1, 1

[1 1 1 1]


**Part 4**

Compare the results obtained from both taggers and a MEMM tagger, implemented by NLTK (a known NLP library), over the test data. To train the NLTK MEMM tagger you should execute the following lines (it may take some time to train...):

In [37]:
from nltk.tag import tnt 

tnt_pos_tagger = tnt.TnT()
tnt_pos_tagger.train(train_data)

nltk_correct_sentense_count = 0.0 

nltk_token_acc =  tnt_pos_tagger.evaluate(test_data)

for sentense in test_data:
    r = tnt_pos_tagger.evaluate([sentense])
    if r == 1:
      nltk_correct_sentense_count += 1

nltk_sentense_acc = nltk_correct_sentense_count/len(test_data)
nltk_token_acc, nltk_sentense_acc

(0.802335803089288, 0.12359550561797752)

Print both, word level and sentence level accuracy for all the three taggers in a table.

In [39]:
# Your code goes here
viterbi_hmm = hmm_tagger()
viterbi_hmm.train(train_data)

simple_hmm = simple_tagger()
simple_hmm.train(train_data)

In [40]:
viterbi_hmm_token_acc, viterbi_hmm_sentense_acc = viterbi_hmm.evaluate(test_data)
simple_hmm_token_acc, simple_hmm_sentense_acc = simple_hmm.evaluate(test_data)

In [42]:
import pandas as pd 
pd.DataFrame([
              ['TnT', nltk_token_acc, nltk_sentense_acc],
              ['hmm_tagger', viterbi_hmm_token_acc, viterbi_hmm_sentense_acc],
              ['simple_hmm', simple_hmm_token_acc, simple_hmm_sentense_acc]
              ], columns=['Tagger', 'Token accuracy', 'Sentence Accuracy'])

,Tagger,Token accuracy,Sentense Accuracy
0,TnT,0.802336,0.123596
1,hmm_tagger,0.809117,0.129213
2,simple_hmm,0.846101,0.189888
